In [0]:
%xmode Minimal
if not 'client' in locals():
    import lakefs_client
    from lakefs_client import models
    from lakefs_client.client import LakeFSClient

    # lakeFS credentials and endpoint
    configuration = lakefs_client.Configuration()
    configuration.username = lakefsAccessKey
    configuration.password = lakefsSecretKey
    configuration.host = lakefsEndPoint

    client = LakeFSClient(configuration)
    print("Created lakeFS client.")

In [0]:
def import_objects(repo, branch, paths, commitMessage, commitMetadata):
    import time

    # Start Import
    import_api = client.__dict__["import"]
    commit = models.CommitCreation(message=commitMessage,metadata=commitMetadata)
    import_creation = models.ImportCreation(paths=paths, commit=commit)
    create_resp = import_api.import_start(repo, branch, import_creation)

    # Wait for import to finish
    while True:
        status_resp = import_api.import_status(repo, branch, create_resp.id)
        print(status_resp)
        if hasattr(status_resp, "Error in import"):
            raise Exception(status_resp.err)
        if status_resp.completed:
            print("Import completed Successfully. Data imported into branch:", branch)
            break
        time.sleep(3)

In [0]:
print("Verifying lakeFS credentials")
client.config.get_lake_fs_version()
print("lakeFS credentials verified")

In [0]:
from lakefs_client.exceptions import NotFoundException

try:
    repo_info=client.repositories.get_repository(repo)
    print(f"Found existing repo {repo_info.id} using storage namespace {repo_info.storage_namespace}")
except NotFoundException as f:
    print(f"Repository {repo} does not exist, so going to try and create it now.")
    try:
        repo_info=client.repositories.create_repository(repository_creation=models.RepositoryCreation(name=repo,
                                                                                                storage_namespace=f"{storageNamespace}"))
        print(f"Created new repo {repo_info.id} using storage namespace {repo_info.storage_namespace}")
    except lakefs_client.ApiException as e:
        print(f"Error creating repo {repo}. Error is {e}")
        os._exit(00)
except lakefs_client.ApiException as e:
    print(f"Error getting repo {repo}: {e}")
    os._exit(00)

In [0]:
client.branches.create_branch(
    repository=repo,
    branch_creation=models.BranchCreation(
        name=emptyBranch,
        source=mainBranch))